<picture>
<a href="https://sambanova.ai/"\>
<source media="(prefers-color-scheme: dark)" srcset="../../images/SambaNova-light-logo-1.png" height="60">
<img alt="SambaNova logo" src="../../images/SambaNova-dark-logo-1.png" height="60">
</picture>
</a>

# Agentops Agent Tracking with SambaNova 

Adapted from the same [Agentops example](https://docs.agentops.ai/v1/examples/multi_agent).

Integrating AgentOps with SambaNova Cloud enhances AI agent development by providing real-time monitoring, debugging, and cost management for LLM inference. AgentOps offers session replay analytics, time-travel debugging, and API latency tracking, helping developers optimize performance and troubleshoot issues efficiently. 

You'll need first to import the necessary modules. Please, create a `.env` file at the repository root and include `SAMBANOVA_URL` and `SAMBANOVA_API_KEY` after creating your [SambaNova Cloud API key](https://cloud.sambanova.ai/apis). You'll also need to add the `AGENTOPS_API_KEY`, so follow the steps in [this link.](https://app.agentops.ai/settings/projects)

Then, create a venv using the framework that you prefer and install the dependencies in the `requirements.txt` located in the current folder.


In [2]:
import os
import agentops
from agentops import track_agent
from openai import OpenAI
from IPython.display import display, Markdown

In [3]:
from dotenv import load_dotenv

load_dotenv('../../.env',override=True)

sambanova_base_url = os.getenv('SAMBANOVA_URL')
sambanova_api_key = os.getenv('SAMBANOVA_API_KEY')
agentops_api_key = os.getenv("AGENTOPS_API_KEY")

import logging
logging.basicConfig(
    level=logging.DEBUG
)  # this will let us see that calls are assigned to an agent

In [4]:
agentops.init(agentops_api_key, default_tags=["multi-agent-notebook"])
sambanova_client = OpenAI(base_url=sambanova_base_url, api_key=sambanova_api_key)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): pypi.org:443
DEBUG:urllib3.connectionpool:https://pypi.org:443 "GET /pypi/agentops/json HTTP/1.1" 200 33610
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_session HTTP/1.1" 200 311
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=83654578-7a8d-4d3a-9674-0de64df3406d


DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/1.1" 200 9


In [5]:
@track_agent(name="qa")
class QaAgent:
    def completion(self, prompt: str):
        res = sambanova_client.chat.completions.create(
            model="Meta-Llama-3.1-70B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a qa engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content


@track_agent(name="engineer")
class EngineerAgent:
    def completion(self, prompt: str):
        res = sambanova_client.chat.completions.create(
            model="Meta-Llama-3.1-70B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a software engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content

In [6]:
qa = QaAgent()
engineer = EngineerAgent()

DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/1.1" 200 9
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/1.1" 200 9


In [7]:
generated_func = engineer.completion("python function to test prime number")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a software engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'python function to test prime number'}], 'model': 'Meta-Llama-3.1-70B-Instruct', 'temperature': 0.5}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.sambanova.ai/v1/chat/completions
DEBUG:httpcore.connection:connect_tcp.started host='api.sambanova.ai' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x105c4b7d0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x107b80c20> server_hostname='api.sambanova.ai' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x107845050>
DEBUG:httpcore.http11:

In [8]:
display(Markdown("```python\n" + generated_func + "\n```"))

```python
def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    max_divisor = int(n**0.5) + 1
    for d in range(3, max_divisor, 2):
        if n % d == 0:
            return False
    return True

def main():
    num = int(input("Enter a number: "))
    if is_prime(num):
        print(num, "is a prime number")
    else:
        print(num, "is not a prime number")

if __name__ == "__main__":
    main()
```

In [9]:
generated_test = qa.completion(
    "Write a python unit test that test the following function: \n " + generated_func
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a qa engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python unit test that test the following function: \n def is_prime(n):\n    if n <= 1:\n        return False\n    if n == 2:\n        return True\n    if n % 2 == 0:\n        return False\n    max_divisor = int(n**0.5) + 1\n    for d in range(3, max_divisor, 2):\n        if n % d == 0:\n            return False\n    return True\n\ndef main():\n    num = int(input("Enter a number: "))\n    if is_prime(num):\n        print(num, "is a prime number")\n    else:\n        print(num, "is not a prime number")\n\nif __name__ == "__main__":\n    main()'}], 'model': 'Meta-Llama-3.1-70B-Instruct', 'temperature': 0.5}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.sambanova.ai/v1/chat/completions
DEBUG:httpcore.h

In [10]:
display(Markdown("```python\n" + generated_test + "\n```"))

```python
```python
import unittest

def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    max_divisor = int(n**0.5) + 1
    for d in range(3, max_divisor, 2):
        if n % d == 0:
            return False
    return True

class TestIsPrimeFunction(unittest.TestCase):
    def test_less_than_or_equal_to_one(self):
        self.assertFalse(is_prime(1))
        self.assertFalse(is_prime(0))
        self.assertFalse(is_prime(-1))

    def test_two(self):
        self.assertTrue(is_prime(2))

    def test_even_numbers(self):
        self.assertFalse(is_prime(4))
        self.assertFalse(is_prime(6))
        self.assertFalse(is_prime(8))

    def test_prime_numbers(self):
        self.assertTrue(is_prime(3))
        self.assertTrue(is_prime(5))
        self.assertTrue(is_prime(7))
        self.assertTrue(is_prime(11))
        self.assertTrue(is_prime(13))

    def test_large_prime_numbers(self):
        self.assertTrue(is_prime(997))
        self.assertTrue(is_prime(1009))

    def test_large_composite_numbers(self):
        self.assertFalse(is_prime(1000))
        self.assertFalse(is_prime(1002))

if __name__ == "__main__":
    unittest.main()
```
```

In [11]:
res = sambanova_client.chat.completions.create(
    model="Meta-Llama-3.1-70B-Instruct",
    messages=[
        {"role": "system", "content": "You are not a tracked agent"},
        {"role": "user", "content": "Say hello"},
    ],
)
res.choices[0].message.content

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are not a tracked agent'}, {'role': 'user', 'content': 'Say hello'}], 'model': 'Meta-Llama-3.1-70B-Instruct'}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.sambanova.ai/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 21 Feb 2025 14:57:57 GMT'), (b'Content-Type', b'application/json; charset=utf-8'), (b'Content-Length', b'866'), (b'Connection', b'keep-alive'), (b'Strict-Transport-Security

"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [12]:
agentops.end_session("Success")

DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/update_session HTTP/1.1" 200 138
🖇 AgentOps: Session Stats - Duration: 4.2s | Cost: $0.00 | LLMs: 3 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=83654578-7a8d-4d3a-9674-0de64df3406d
